In [1]:
from datasets import *
import pandas as pd
from matplotlib import pyplot as plt

/Users/ebony/miniforge3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset('ebony59/gsm8k-gen-sel', split='train')

Generating train split: 100%|██████████| 57332/57332 [00:00<00:00, 626791.25 examples/s]


In [8]:
df = dataset.to_pandas()
df.head(10)

,question,gt_answer,orig_answer,llm_answer,correct,distance
0,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,0
1,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,15
2,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,45
3,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,105
4,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,225
5,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,465
6,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,945
7,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,1905
8,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,3825
9,Natalia sold clips to 48 of her friends in Apr...,72,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,Natalia sold 48/2 = <<48/2=24>>24 clips in Ma...,True,7665


In [9]:
from validate_data import *

In [13]:
orig_equations = get_equations(df.loc[2, 'orig_answer'])
equations = get_equations(df.loc[2, 'llm_answer'])

print(orig_equations)
print(equations)

['48/2=24', '48+24=72']
['48/2=24', '48+24=72']


In [14]:
Levenshtein.distance('|'.join(orig_equations), '|'.join(equations))

0

In [22]:
small_ds = df.head(20).to_dict(orient='records')

In [ ]:
def temp_get_distances(ds):
    curr_question = ds[0]['question']
    orig_equations = get_equations(ds[0]['orig_answer'])

    selected_ds = []

    for d in tqdm(ds):
        question = d['question']
        if question != curr_question:
            curr_question = question
            orig_equations = get_equations(d['orig_answer'])

            print(curr_question, d['orig_answer'])

            if len(orig_equations) == 0:
                orig_equations = ['None']

        if d['correct'] == False:
            continue

        equations = get_equations(d['llm_answer'])
        if validate_equations(equations) == False:
            continue

        if len(equations) == 0:
            equations = ['None']

        # print('equations', equations)

        str_equations = '|'.join(equations)
        str_orig_equations = '|'.join(orig_equations)

        # print(equations, orig_equations)
        dist = Levenshtein.distance(str_equations, str_orig_equations)
        dist = dist / len(orig_equations)

        # dist = 0
        # counts = 0
        # for i, equation_i in enumerate(equations):
        #     if equation_i == orig_equations[i]:
        #         continue
        #     for j, equation_j in enumerate(orig_equations):
        #         dist += Levenshtein.distance(equation_i, equation_j)
        # dist = dist / (len(equations)*len(orig_equations))
        d['distance'] = dist
        selected_ds.append(d)

    return selected_ds

In [24]:
temp_get_distances(small_ds)

100%|██████████| 20/20 [00:00<00:00, 21167.32it/s]

Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet? In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.
Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.
This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.
#### 5


[{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
  'gt_answer': '72',
  'orig_answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72',
  'llm_answer': ' Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72',
  'correct': True,
  'distance': 0.0},
 {'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
  'gt_answer': '72',
  'orig_answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72',
  'llm_answer': ' Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in Apr